## Common Models

Some models are common to all components. While these common models are the same in almost every respect these models do require an extra argument to define the "emitter" (``"stellar"``, ``"blackhole"``, or ``"galaxy"``) that they act on.

### TemplateEmission

A template is a simple generation model where a single template spectra is returned with some scaling applying. These can describe any emitter based on the spectra input into the template (which can either come from arrays of ``lnu`` and ``lam`` or a file). 

Generation from a template model requires a ``Template``, similar to the ``Grid`` required by extraction models.

In [1]:
import numpy as np
from unyt import Hz, erg, kelvin, s

from synthesizer.emission_models import TemplateEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody
from synthesizer.grid import Grid, Template

# Get the grid which we'll need for extraction
grid_dir = "../../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Make a fake template for the demo
template = Template(
    lam=grid.lam, lnu=np.random.rand(*grid.lam.shape) * erg / s / Hz
)

template_model = TemplateEmission(template, emitter="stellar")
print(template_model)

|================================= EmissionModel: template =================================|
|-------------------------------------------------------------------------------------------|
|  TEMPLATE (stellar)                                                                       |
|-------------------------------------------------------------------------------------------|
|Generation model:                                                                          |
|  Emission generation model: <synthesizer.grid.Template object at 0x7fd3f073b340>          |
|  Save emission: True                                                                      |
|===========================================================================================|


### AttenuatedEmission

Attenuated emission is any emission that has been attenuated by dust.
An attenuated model takes a ``dust_curve`` (with a defined slope), an optical depth (``tau_v``, which can either be a value, array, attribute of an emitter, or a tuple containing any combination of the first 3), and a model to apply the dust attenuation to (``apply_dust_to``).
Here we create a simple Intrinsic emission model to which we apply the dust attenuation.

In [2]:
from synthesizer.emission_models import AttenuatedEmission, IntrinsicEmission

intrinsic = IntrinsicEmission(grid, fesc=0.1, fesc_ly_alpha=1.0)

attenuated = AttenuatedEmission(
    emitter="stellar",
    dust_curve=PowerLaw(slope=-1),
    tau_v=(0.5, "tau_v"),
    apply_dust_to=intrinsic,
)
print(attenuated)

|====================================== EmissionModel: attenuated =======================================|
|--------------------------------------------------------------------------------------------------------|
|  ESCAPED (stellar)                                                                                     |
|--------------------------------------------------------------------------------------------------------|
|Extraction model:                                                                                       |
|  Grid: test_grid                                                                                       |
|  Extract key: transmitted                                                                              |
|  Escape fraction: 0.9                                                                                  |
|  Save emission: True                                                                                   |
|------------------------------------

### DustEmission

Dust emission is the reemited emission from the dust distribution due to the attenuation of another emission source. A dust emission model defines the generation of a spectra from a generator (e.g. ``Blackbody`` or ``Greybody``, see the [dust emission model docs](../dust_emission.ipynb)) and it its scaling using an energy balance approach.

In [3]:
from synthesizer.emission_models import DustEmission

dust_model = DustEmission(
    dust_emission_model=Blackbody(50 * kelvin),
    dust_lum_intrinsic=intrinsic,
    dust_lum_attenuated=attenuated,
    emitter="stellar",
)
print(dust_model)

|============================================ EmissionModel: dust_emission ===========================================|
|---------------------------------------------------------------------------------------------------------------------|
|  ESCAPED (stellar)                                                                                                  |
|---------------------------------------------------------------------------------------------------------------------|
|Extraction model:                                                                                                    |
|  Grid: test_grid                                                                                                    |
|  Extract key: transmitted                                                                                           |
|  Escape fraction: 0.9                                                                                               |
|  Save emission: True                  